In [1]:
#Importation des librairies
import warnings
warnings.filterwarnings("ignore")

import os
import re
import pandas as pd
import missingno as msno
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import nltk
import nlp_module
import string,unicodedata
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from nlp_module import remove_stopwords, plot_top_words, tok, print_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
import ast

from string import punctuation
from nltk.corpus import stopwords
from time import time

pd.set_option("display.max_columns", None)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("df.csv", sep=',')

In [3]:
df = df.sample(frac=0.1,random_state=1)

In [4]:
df = df.reset_index()

In [5]:
df

,index,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,Tags_cpt,Tags1,Tags2,Tags3,Tags4,Tags5
0,96087,Why chart.js canvas respecting padding contain...,I using Chart.js simple line chart width heigh...,"['javascript', 'html', 'css', 'canvas', 'chart...",28162810,28,51607,6.0,6,5,python,None,None,None,None
1,67627,xcode 'boost/config/user.hpp' file found,"I' using Xcode 9, everything fine. I upgrade R...","['ios', 'xcode', 'boost', 'react-native', 'npm']",45262978,15,8915,4.0,5,5,javascript,jquery,css,angularjs,twitter-bootstrap
2,35382,Rotate marker Leaflet,"How I rotate marker leaflet? I lot markers, ro...","['javascript', 'rotation', 'icons', 'marker', ...",13494649,17,25926,7.0,4,5,javascript,None,None,None,None
3,13155,Xcode 10 archiving fails - command line (xcode...,"I' issues archiving CI machine (Jenkins), runn...","['ios', 'cocoapods', 'xcodebuild', 'xcode10', ...",52687998,29,26728,9.0,5,5,ios,None,None,None,None
4,98019,List.RemoveAll() efficiency / compiler optimis...,"Regarding efficiency, anyone know compiler cle...","['c#', 'arrays', 'list', 'linq', 'compiler-opt...",55531327,26,1384,4.0,3,5,c#,.net,security,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,53530,Reference Volume/Drive Label,I' trying write batch file xcopy folder remova...,"['batch-file', 'command', 'label', 'volume', '...",9065280,6,18774,2.0,3,5,linux,unix,None,None,None
10488,88560,Flutter - Validate phone number using Regex,"In Flutter mobile app, I trying validate phone...","['android', 'ios', 'regex', 'dart', 'flutter']",55552230,38,48634,12.0,5,5,javascript,node.js,typescript,express,None
10489,36690,Spring Boot + Eureka Server + Hystrix Turbine:...,I' trying run Spring Boot ( Spring Cloud) + Eu...,"['spring-boot', 'groovy', 'spring-cloud', 'hys...",28365202,22,16246,8.0,2,5,spring-boot,None,None,None,None
10490,86184,What difference protected private?,I ' understand difference protected private me...,"['c++', 'oop', 'class', 'private', 'protected']",12784083,43,102862,10.0,5,5,java,spring,spring-boot,None,None


In [6]:
df.isnull().sum().sum()

0

In [7]:
df = df.dropna()

In [8]:
df_sample = df.sample(frac=0.10, random_state=1)

In [9]:
#temporary for faster test
full_text = df_sample["Title"].values + " " + df_sample["Body"].values


df["full_text"] = df["Title"] + df["Body"]

In [10]:
bow_vectorizer = CountVectorizer(ngram_range = (1,1),
                                 tokenizer=tok)
text_bow = bow_vectorizer.fit_transform(full_text)

In [11]:
lda = LatentDirichletAllocation(random_state=42,
                                batch_size=400)

params = {'n_components' : [8, 9, 10],
          "learning_decay" : [0.5, 0.7, 0.9],
          "learning_method" : ["batch", "online"]}

gridsearch_lda = GridSearchCV(lda,
                              param_grid=params,
                              cv=5,
                              verbose=2)
#gridsearch_lda.fit(text_bow)

In [12]:
#best_lda_model = gridsearch_lda.best_estimator_

#print("Best Model's Params: ", gridsearch_lda.best_params_)
#print("Best Log Likelihood Score: ", gridsearch_lda.best_score_)

In [13]:
# Visualize the topics
#pyLDAvis.enable_notebook()
#panel = pyLDAvis.sklearn.prepare(best_lda_model, text_bow, bow_vectorizer, mds='tsne')
#panel

In [14]:
df["Tags1"].describe()

count     10492
unique      134
top        java
freq       1247
Name: Tags1, dtype: object

In [15]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

ohe_df = pd.DataFrame(ohe.fit_transform(df[['Tags1','Tags2', 'Tags3']]))

In [16]:
one_hot1 = pd.get_dummies(df["Tags1"],
                         prefix="",
                         drop_first=False)

one_hot2 = pd.get_dummies(df["Tags2"],
                         prefix="",
                         drop_first=False)

one_hot3 = pd.get_dummies(df["Tags3"],
                         prefix="",
                         drop_first=False)

one_hot4 = pd.get_dummies(df["Tags4"],
                         prefix="",
                         drop_first=False)

one_hot5 = pd.get_dummies(df["Tags5"],
                         prefix="",
                         drop_first=False)

In [17]:
one_hot1

,_.net,_None,_ajax,_algorithm,_amazon-web-services,_android,_android-studio,_angular,_angularjs,_apache,_apache-spark,_api,_arrays,_asp.net,_asp.net-core,_asp.net-mvc,_asp.net-web-api,_assembly,_asynchronous,_authentication,_bash,_c,_c#,_c++,_c++11,_class,_cocoa,_cocoa-touch,_concurrency,_css,_data-structures,_database,_date,_datetime,_debugging,_dictionary,_django,_docker,_eclipse,_entity-framework,_file,_firebase,_function,_gcc,_git,_google-chrome,_gradle,_haskell,_hibernate,_html,_http,_image,_image-processing,_inheritance,_ios,_ipad,_iphone,_java,_java-8,_javascript,_jpa,_jquery,_json,_laravel,_linq,_linux,_list,_logging,_machine-learning,_macos,_math,_maven,_memory,_memory-management,_mongodb,_multithreading,_mysql,_node.js,_numpy,_objective-c,_oop,_opencv,_optimization,_orm,_pandas,_parsing,_performance,_php,_postgresql,_python,_python-2.7,_python-3.x,_qt,_r,_reactjs,_regex,_rest,_ruby,_ruby-on-rails,_ruby-on-rails-3,_scala,_security,_selenium,_shell,_sockets,_sorting,_spring,_spring-boot,_spring-mvc,_sql,_sql-server,_ssl,_string,_swift,_templates,_testing,_twitter-bootstrap,_types,_typescript,_ubuntu,_uitableview,_unicode,_unit-testing,_unix,_user-interface,_validation,_visual-studio,_visual-studio-2010,_web-services,_windows,_wpf,_x86,_xcode,_xml
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10490,0,0,0,0

In [18]:
one_hot2

,_.net,_None,_ajax,_algorithm,_amazon-web-services,_android,_android-layout,_android-studio,_angular,_angularjs,_apache,_apache-spark,_api,_arrays,_asp.net,_asp.net-core,_asp.net-mvc,_asp.net-web-api,_assembly,_async-await,_asynchronous,_authentication,_bash,_c,_c#,_c++,_c++11,_class,_cocoa,_cocoa-touch,_concurrency,_css,_data-structures,_database,_dataframe,_date,_datetime,_debugging,_dictionary,_django,_docker,_eclipse,_entity-framework,_exception,_express,_file,_firebase,_forms,_function,_gcc,_generics,_git,_google-chrome,_gradle,_haskell,_hibernate,_html,_http,_image,_image-processing,_inheritance,_ios,_ipad,_iphone,_java,_java-8,_javascript,_jpa,_jquery,_json,_lambda,_language-lawyer,_laravel,_linq,_linux,_list,_logging,_machine-learning,_macos,_math,_matplotlib,_maven,_memory,_memory-management,_mongodb,_multithreading,_mysql,_node.js,_npm,_numpy,_objective-c,_oop,_opencv,_optimization,_orm,_pandas,_parsing,_performance,_php,_pointers,_postgresql,_python,_python-2.7,_python-3.x,_qt,_r,_reactjs,_regex,_rest,_ruby,_ruby-on-rails,_ruby-on-rails-3,_scala,_security,_selenium,_shell,_sockets,_sorting,_spring,_spring-boot,_spring-mvc,_sql,_sql-server,_ssl,_string,_swift,_templates,_testing,_tsql,_twitter-bootstrap,_types,_typescript,_ubuntu,_uitableview,_unicode,_unit-testing,_unix,_user-interface,_validation,_visual-studio,_visual-studio-2010,_web-services,_windows,_winforms,_wpf,_x86,_xaml,_xcode,_xml
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
one_hot3

,_.net,_None,_ajax,_algorithm,_amazon-web-services,_android,_android-layout,_android-studio,_angular,_angularjs,_apache,_apache-spark,_api,_arrays,_asp.net,_asp.net-core,_asp.net-mvc,_asp.net-web-api,_assembly,_async-await,_asynchronous,_authentication,_bash,_c,_c++,_c++11,_class,_cocoa,_cocoa-touch,_concurrency,_css,_data-structures,_database,_dataframe,_date,_datetime,_debugging,_dictionary,_django,_docker,_eclipse,_entity-framework,_exception,_express,_file,_firebase,_forms,_function,_gcc,_generics,_git,_google-chrome,_gradle,_haskell,_hibernate,_html,_http,_image,_image-processing,_inheritance,_ios,_ipad,_iphone,_java,_java-8,_javascript,_jpa,_jquery,_json,_lambda,_language-lawyer,_laravel,_linq,_linux,_list,_logging,_machine-learning,_macos,_math,_matplotlib,_maven,_memory,_memory-management,_mongodb,_multithreading,_mysql,_node.js,_npm,_numpy,_objective-c,_oop,_opencv,_optimization,_orm,_pandas,_parsing,_performance,_php,_pointers,_postgresql,_python,_python-2.7,_python-3.x,_qt,_r,_reactjs,_regex,_rest,_ruby,_ruby-on-rails,_ruby-on-rails-3,_scala,_security,_selenium,_shell,_sockets,_sorting,_spring,_spring-boot,_spring-mvc,_sql,_sql-server,_ssl,_string,_swift,_templates,_testing,_tsql,_twitter-bootstrap,_types,_typescript,_ubuntu,_uitableview,_unicode,_unit-testing,_unix,_user-interface,_validation,_visual-studio,_visual-studio-2010,_web-services,_windows,_winforms,_wpf,_x86,_xaml,_xcode,_xml
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
one_hot4

,_.net,_None,_ajax,_algorithm,_amazon-web-services,_android,_android-layout,_android-studio,_angular,_angularjs,_apache,_apache-spark,_api,_arrays,_asp.net,_asp.net-core,_asp.net-mvc,_asp.net-web-api,_assembly,_async-await,_asynchronous,_authentication,_c,_c++,_c++11,_class,_cocoa,_cocoa-touch,_concurrency,_css,_data-structures,_database,_dataframe,_date,_datetime,_debugging,_dictionary,_django,_docker,_eclipse,_entity-framework,_exception,_express,_file,_firebase,_forms,_function,_gcc,_generics,_google-chrome,_gradle,_haskell,_hibernate,_html,_http,_image,_image-processing,_inheritance,_ios,_ipad,_iphone,_java-8,_jpa,_jquery,_json,_lambda,_language-lawyer,_laravel,_linq,_linux,_list,_logging,_machine-learning,_macos,_math,_matplotlib,_maven,_memory,_memory-management,_multithreading,_mysql,_node.js,_npm,_numpy,_oop,_opencv,_optimization,_orm,_pandas,_parsing,_performance,_php,_pointers,_postgresql,_python-2.7,_python-3.x,_qt,_reactjs,_regex,_rest,_ruby,_ruby-on-rails-3,_security,_selenium,_shell,_sockets,_sorting,_spring,_spring-boot,_spring-mvc,_sql,_sql-server,_ssl,_string,_swift,_templates,_testing,_tsql,_twitter-bootstrap,_types,_typescript,_ubuntu,_uitableview,_unicode,_unit-testing,_unix,_user-interface,_validation,_visual-studio,_visual-studio-2010,_web-services,_windows,_winforms,_x86,_xaml,_xcode,_xml
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10489,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [21]:
one_hot5

,_.net,_None,_ajax,_algorithm,_android-studio,_angularjs,_arrays,_asp.net-core,_asp.net-mvc,_asp.net-web-api,_assembly,_async-await,_asynchronous,_authentication,_c,_c++11,_cocoa,_cocoa-touch,_concurrency,_css,_data-structures,_database,_dataframe,_date,_datetime,_debugging,_dictionary,_django,_docker,_eclipse,_entity-framework,_express,_firebase,_forms,_gcc,_generics,_git,_google-chrome,_gradle,_http,_image,_image-processing,_inheritance,_ios,_ipad,_java-8,_jpa,_jquery,_json,_lambda,_language-lawyer,_laravel,_linq,_logging,_machine-learning,_macos,_maven,_memory,_memory-management,_multithreading,_node.js,_npm,_numpy,_optimization,_orm,_pandas,_parsing,_performance,_pointers,_postgresql,_python-2.7,_python-3.x,_rest,_ruby,_security,_selenium,_shell,_spring-boot,_spring-mvc,_sql,_ssl,_string,_swift,_templates,_tsql,_twitter-bootstrap,_types,_typescript,_ubuntu,_uitableview,_unicode,_unix,_user-interface,_validation,_visual-studio,_winforms,_x86,_xcode
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10488,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10489,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10490,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [23]:
matrix = tf.fit_transform(df['full_text'])

In [24]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(matrix,matrix)

In [25]:
post_title = df['Title']

indices = pd.Series(df.index, index=df['Title'])

In [26]:
def post_recommend(original_title):
    
    
    
    idx = indices[original_title]

    sim_scores = list(enumerate(cosine_similarities[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:31]

    movie_indices = [i[0] for i in sim_scores]

    return df.iloc[movie_indices]

In [27]:
def recommend_tags(original_title, sample_size):
    posts = post_recommend(original_title).head(sample_size)
    
    tags = posts["Tags"].apply(ast.literal_eval).apply(lambda x: pd.Series(x)).stack().value_counts()
    return tags

In [28]:
post = recommend_tags('What difference protected private?',100)

In [29]:
post

java                      8
c#                        7
private                   4
c++                       4
access-modifiers          3
                         ..
final                     1
erlang                    1
functional-programming    1
static                    1
numeric                   1
Length: 111, dtype: int64

In [30]:
post

java                      8
c#                        7
private                   4
c++                       4
access-modifiers          3
                         ..
final                     1
erlang                    1
functional-programming    1
static                    1
numeric                   1
Length: 111, dtype: int64

In [31]:
tags = post.index.tolist()

In [32]:
tags[:5]

['java', 'c#', 'private', 'c++', 'access-modifiers']